# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

# Eager mode on

In [2]:
tf.enable_eager_execution()

# Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# Create model (class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

# Create model (function version)

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

# Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.3115, train_loss: 2.14351797, train_accuracy: 0.4922, test_Accuracy: 0.2823
Epoch: [ 0] [    1/  468] time: 0.4366, train_loss: 2.10850906, train_accuracy: 0.4922, test_Accuracy: 0.4468
Epoch: [ 0] [    2/  468] time: 0.5389, train_loss: 2.01465869, train_accuracy: 0.6250, test_Accuracy: 0.5340
Epoch: [ 0] [    3/  468] time: 0.6307, train_loss: 2.00693560, train_accuracy: 0.5469, test_Accuracy: 0.5937
Epoch: [ 0] [    4/  468] time: 0.7357, train_loss: 1.88391960, train_accuracy: 0.6172, test_Accuracy: 0.6357
Epoch: [ 0] [    5/  468] time: 0.8431, train_loss: 1.78965557, train_accuracy: 0.6406, test_Accuracy: 0.6713
Epoch: [ 0] [    6/  468] time: 0.9512, train_loss: 1.70328987, train_accuracy: 0.6953, test_Accuracy: 0.6985
Epoch: [ 0] [    7/  468] time: 1.0521, train_loss: 1.52768254, train_accuracy: 0.6875, test_Accuracy: 0.7223
Epoch: [ 0] [    8/  468] time: 1.1572,

Epoch: [ 0] [   76/  468] time: 7.5749, train_loss: 0.25014365, train_accuracy: 0.9141, test_Accuracy: 0.9081
Epoch: [ 0] [   77/  468] time: 7.6634, train_loss: 0.29067466, train_accuracy: 0.8984, test_Accuracy: 0.9093
Epoch: [ 0] [   78/  468] time: 7.7502, train_loss: 0.40782946, train_accuracy: 0.8828, test_Accuracy: 0.9081
Epoch: [ 0] [   79/  468] time: 7.8430, train_loss: 0.29894257, train_accuracy: 0.8750, test_Accuracy: 0.9082
Epoch: [ 0] [   80/  468] time: 7.9286, train_loss: 0.29954958, train_accuracy: 0.9141, test_Accuracy: 0.9065
Epoch: [ 0] [   81/  468] time: 8.0173, train_loss: 0.48582801, train_accuracy: 0.8281, test_Accuracy: 0.9082
Epoch: [ 0] [   82/  468] time: 8.1165, train_loss: 0.28034693, train_accuracy: 0.9219, test_Accuracy: 0.9093
Epoch: [ 0] [   83/  468] time: 8.2144, train_loss: 0.39410660, train_accuracy: 0.8828, test_Accuracy: 0.9134
Epoch: [ 0] [   84/  468] time: 8.3135, train_loss: 0.19375709, train_accuracy: 0.9375, test_Accuracy: 0.9151
Epoch: [ 0

Epoch: [ 0] [  153/  468] time: 14.7648, train_loss: 0.22450532, train_accuracy: 0.9219, test_Accuracy: 0.9282
Epoch: [ 0] [  154/  468] time: 14.8588, train_loss: 0.21915072, train_accuracy: 0.9375, test_Accuracy: 0.9320
Epoch: [ 0] [  155/  468] time: 14.9522, train_loss: 0.18834199, train_accuracy: 0.9375, test_Accuracy: 0.9321
Epoch: [ 0] [  156/  468] time: 15.0483, train_loss: 0.18547265, train_accuracy: 0.9453, test_Accuracy: 0.9305
Epoch: [ 0] [  157/  468] time: 15.1411, train_loss: 0.28770554, train_accuracy: 0.9375, test_Accuracy: 0.9301
Epoch: [ 0] [  158/  468] time: 15.2347, train_loss: 0.29708147, train_accuracy: 0.9297, test_Accuracy: 0.9297
Epoch: [ 0] [  159/  468] time: 15.3289, train_loss: 0.21299247, train_accuracy: 0.9531, test_Accuracy: 0.9288
Epoch: [ 0] [  160/  468] time: 15.4235, train_loss: 0.15039845, train_accuracy: 0.9688, test_Accuracy: 0.9295
Epoch: [ 0] [  161/  468] time: 15.5116, train_loss: 0.16528589, train_accuracy: 0.9453, test_Accuracy: 0.9300
E

Epoch: [ 0] [  228/  468] time: 21.7107, train_loss: 0.10918133, train_accuracy: 0.9609, test_Accuracy: 0.9433
Epoch: [ 0] [  229/  468] time: 21.8019, train_loss: 0.16011849, train_accuracy: 0.9375, test_Accuracy: 0.9443
Epoch: [ 0] [  230/  468] time: 21.8965, train_loss: 0.25482100, train_accuracy: 0.9375, test_Accuracy: 0.9441
Epoch: [ 0] [  231/  468] time: 21.9840, train_loss: 0.29531652, train_accuracy: 0.9219, test_Accuracy: 0.9442
Epoch: [ 0] [  232/  468] time: 22.0775, train_loss: 0.16539118, train_accuracy: 0.9453, test_Accuracy: 0.9436
Epoch: [ 0] [  233/  468] time: 22.1711, train_loss: 0.22030769, train_accuracy: 0.9453, test_Accuracy: 0.9446
Epoch: [ 0] [  234/  468] time: 22.2616, train_loss: 0.20346518, train_accuracy: 0.9375, test_Accuracy: 0.9454
Epoch: [ 0] [  235/  468] time: 22.3556, train_loss: 0.19502532, train_accuracy: 0.9375, test_Accuracy: 0.9471
Epoch: [ 0] [  236/  468] time: 22.4494, train_loss: 0.19169007, train_accuracy: 0.9297, test_Accuracy: 0.9459
E

Epoch: [ 0] [  303/  468] time: 28.6971, train_loss: 0.17693420, train_accuracy: 0.9375, test_Accuracy: 0.9508
Epoch: [ 0] [  304/  468] time: 28.7939, train_loss: 0.19185643, train_accuracy: 0.9375, test_Accuracy: 0.9505
Epoch: [ 0] [  305/  468] time: 28.8880, train_loss: 0.22893812, train_accuracy: 0.9453, test_Accuracy: 0.9489
Epoch: [ 0] [  306/  468] time: 28.9874, train_loss: 0.10493398, train_accuracy: 0.9688, test_Accuracy: 0.9488
Epoch: [ 0] [  307/  468] time: 29.0775, train_loss: 0.17147568, train_accuracy: 0.9609, test_Accuracy: 0.9486
Epoch: [ 0] [  308/  468] time: 29.1705, train_loss: 0.16677657, train_accuracy: 0.9531, test_Accuracy: 0.9485
Epoch: [ 0] [  309/  468] time: 29.2631, train_loss: 0.12627694, train_accuracy: 0.9766, test_Accuracy: 0.9490
Epoch: [ 0] [  310/  468] time: 29.3583, train_loss: 0.19799855, train_accuracy: 0.9453, test_Accuracy: 0.9502
Epoch: [ 0] [  311/  468] time: 29.4523, train_loss: 0.11391820, train_accuracy: 0.9766, test_Accuracy: 0.9517
E

Epoch: [ 0] [  378/  468] time: 35.7245, train_loss: 0.13656595, train_accuracy: 0.9609, test_Accuracy: 0.9560
Epoch: [ 0] [  379/  468] time: 35.8200, train_loss: 0.11856002, train_accuracy: 0.9609, test_Accuracy: 0.9545
Epoch: [ 0] [  380/  468] time: 35.9142, train_loss: 0.16988833, train_accuracy: 0.9453, test_Accuracy: 0.9546
Epoch: [ 0] [  381/  468] time: 36.0067, train_loss: 0.14984861, train_accuracy: 0.9453, test_Accuracy: 0.9559
Epoch: [ 0] [  382/  468] time: 36.1010, train_loss: 0.19075733, train_accuracy: 0.9453, test_Accuracy: 0.9578
Epoch: [ 0] [  383/  468] time: 36.1958, train_loss: 0.13548537, train_accuracy: 0.9609, test_Accuracy: 0.9576
Epoch: [ 0] [  384/  468] time: 36.2906, train_loss: 0.33538210, train_accuracy: 0.9062, test_Accuracy: 0.9574
Epoch: [ 0] [  385/  468] time: 36.3815, train_loss: 0.15151492, train_accuracy: 0.9531, test_Accuracy: 0.9585
Epoch: [ 0] [  386/  468] time: 36.4774, train_loss: 0.24722907, train_accuracy: 0.9609, test_Accuracy: 0.9592
E

Epoch: [ 0] [  453/  468] time: 42.7963, train_loss: 0.14623877, train_accuracy: 0.9531, test_Accuracy: 0.9603
Epoch: [ 0] [  454/  468] time: 42.8869, train_loss: 0.28326234, train_accuracy: 0.8984, test_Accuracy: 0.9609
Epoch: [ 0] [  455/  468] time: 42.9810, train_loss: 0.09695946, train_accuracy: 0.9688, test_Accuracy: 0.9607
Epoch: [ 0] [  456/  468] time: 43.0790, train_loss: 0.09337467, train_accuracy: 0.9844, test_Accuracy: 0.9595
Epoch: [ 0] [  457/  468] time: 43.1805, train_loss: 0.12247287, train_accuracy: 0.9688, test_Accuracy: 0.9600
Epoch: [ 0] [  458/  468] time: 43.2767, train_loss: 0.13066933, train_accuracy: 0.9609, test_Accuracy: 0.9608
Epoch: [ 0] [  459/  468] time: 43.3719, train_loss: 0.09534849, train_accuracy: 0.9688, test_Accuracy: 0.9614
Epoch: [ 0] [  460/  468] time: 43.4639, train_loss: 0.06053156, train_accuracy: 0.9922, test_Accuracy: 0.9622
Epoch: [ 0] [  461/  468] time: 43.5593, train_loss: 0.10711938, train_accuracy: 0.9531, test_Accuracy: 0.9616
E

# Test accuracy : 96.22 %